In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [35]:
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from common.helper import make_absolute_urls
import requests
import html2text
import validators


base_url = "https://your.yale.edu"
policies_url = f"{base_url}/policies-procedures/policies"

def process_policy_document(policy):
    policy_text = str(policy)
    s = BeautifulSoup(policy_text)
    heading = s.find("h1")
    if heading is None: 
        return 
    title = heading.text.strip()
    text = html2text.html2text(policy_text)
    return (title, text)
    
page = requests.get(policies_url)
soup = BeautifulSoup(page.text)

# Scrape policy links from webpage
policy_toc = soup.find("tbody")
policy_link_tags = policy_toc.findChildren("a",href=True)
policy_links = make_absolute_urls([el["href"] for el in policy_link_tags], base_url)


# # Scrape content from each link
policy_list = []
for policy_section in policy_links:
        try:
            print(policy_section)
            policy_page = requests.get(policy_section)
            soup = BeautifulSoup(policy_page.text)
            policy_content = soup.find("div", class_="main")
            policy_list.append(process_policy_document(policy_content))
        except:
            print("Encountered error.")

print(f"Scraped {len(policy_list)} policy from {policies_url}.")

https://your.yale.edu/policies-procedures/policies/1000-development-revision-and-posting-university-policies-procedures
https://your.yale.edu/policies-procedures/policies/1001-compensation-above-salary
https://your.yale.edu/policies-procedures/policies/1010-training-and-other-requirements
https://your.yale.edu/policies-procedures/policies/1101-guiding-principles-business-and-financial-administration
https://your.yale.edu/policies-procedures/policies/1104-signature-authority-approval-authority-and-access-financial
https://your.yale.edu/policies-procedures/policies/1105-retention-university-financial-records
https://your.yale.edu/policies-procedures/policies/1106-preserving-records-legal-purposes
https://your.yale.edu/policies-procedures/policies/1109-internet-access-home
https://your.yale.edu/policies-procedures/policies/1110-personal-and-campus-use-university-property-including-vehicles
https://your.yale.edu/policies-procedures/policies/120-irb-investigator-initiated-and-multicenter-re

In [41]:
from common.helper import save_policies
save_policies(policy_list,"/Users/erikbeerepoot//Documents/policies/yale")